<a href="https://colab.research.google.com/github/cowsilver57/sessac_test/blob/main/Chapter8_1123_%EB%AA%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#github와 colab 연동
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 637, done.
remote: Counting objects: 100% (276/276), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 637 (delta 137), reused 184 (delta 109), pack-reused 361
Receiving objects: 100% (637/637), 10.76 MiB | 13.54 MiB/s, done.
Resolving deltas: 100% (321/321), done.


#1유형

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv",index_col=0)
df.head()

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01
2,2020년 제야의 종 온라인 타종행사 | 보신각 현장 행사는 진행하지 않습니다.,서울시 · Seoul,29,347049,3564,120,178,UCZUPZW5idAxYp-Asj__lVAA,2021-01-01
3,고기남자의 칠면조 파티,고기남자 MeatMan,26,528458,15372,280,3470,UCT3CumbFIJiW33uq0UI3zlg,2021-01-01
4,골목 3mc를 분노하게 만든 마음고생이 심했을 공릉 백반집 사장님의 푸념?! [예능...,스브스밥집,24,494904,3918,111,3142,UCdWgRSfttvDucq4ApcCg5Mw,2021-01-01


1번 인기동영상 제작횟수가 많은 채널 상위 10개명을 출력하라(날짜기준, 중복포함)

In [ ]:
print(list(df.loc[df.channelId.isin(df.channelId.value_counts().head(10).index)].channelTitle.unique()))

['장삐쭈', '총몇명', '파뿌리', '짤툰', '런닝맨 - 스브스 공식 채널', '엠뚜루마뚜루 : MBC 공식 종합 채널', 'SPOTV', '채널 십오야', '이과장', 'BANGTANTV']


2번 논란으로 인기동영상이 된 케이스를 확인하고 싶다. dislikes수가 like수보다 높은 동영상을 제작한 채널을 모두 출력하라

In [ ]:
result = list((df.loc[df.likes<df.dislikes]).channelTitle.unique())
print(result)

['핫도그TV', 'ASMR 애정TV', '하얀트리HayanTree', '양팡 YangPang', '철구형 (CHULTUBE)', '왜냐맨하우스', '(MUTUBE)와꾸대장봉준', '오메킴TV', '육지담', 'MapleStory_KR', 'ROAD FIGHTING CHAMPIONSHIP', '사나이 김기훈', '나혼자산다 STUDIO', 'Gen.G esports']


3번 채널명을 바꾼 케이스가 있는지 확인하고 싶다. channelId의 경우 고유값이므로 이를 통해 채널명을 한번이라도 바꾼 채널의 갯수를 구하여라

In [ ]:
change = df[['channelTitle', 'channelId']].drop_duplicates().channelId.value_counts()
result = len(change[change>1])
print(result)

71


4번 일요일에 인기있었던 영상들중 가장많은 영상 종류(categoryId)는 무엇인가?

In [ ]:
df['trending_date2'] = pd.to_datetime(df['trending_date2'])
result = df.loc[df['trending_date2'].dt.day_name()=='Sunday'].categoryId.value_counts().index[0]
print(result)

24


5번 각 요일별 인기 영상들의 categoryId는 각각 몇개 씩인지 하나의 데이터 프레임으로 표현하라

In [ ]:
print(df['trending_date2'].dtype)
df['trending_date2'] = pd.to_datetime(df['trending_date2'])
group = df.groupby([df['trending_date2'].dt.day_name(),'categoryId'],as_index=False).size()
answer= group.pivot(index='categoryId',columns='trending_date2')
display(answer)

object


size                                                  
trending_date2 Friday Monday Saturday Sunday Thursday Tuesday Wednesday
categoryId                                                             
1                 243    263      255    274      246     257       234
2                 120    105      119     99      128     119       129
10                833    837      776    830      890     894       917
15                187    215      198    217      207     208       207
17                633    668      592    636      682     708       706
19                 90     92       87     91       92      89        85
20                283    298      296    289      282     285       291
22               1288   1373     1289   1337     1341    1375      1333
23                568    594      570    556      560     569       566
24               2976   3148     3066   3096     2954    3084      3090
25                444    453      422    437      470     452       468
26                369    378      364    363      375     394       385
27                183    205      183    199      194     194       212
28                171    160      173    167      166     161       165
29                 12     10       10      9       13      11        12

6번 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다. viewcount대비 댓글수가 가장 높은 영상을 확인하라 (view_count값이 0인 경우는 제외한다)

In [ ]:
target2= df.loc[df.view_count!=0]
t = target2.copy()
t['ratio'] = (target2['comment_count']/target2['view_count']).dropna()
result = t.sort_values(by='ratio', ascending=False).iloc[0].title
print(result)

60분 동안 댓글이 달리지 않으면, 영상이 삭제됩니다. (챌린지)


7번 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다.viewcount대비 댓글수가 가장 낮은 영상을 확인하라 (view_counts, ratio값이 0인경우는 제외한다.)

In [ ]:
ratio = (df['comment_count']/df['view_count']).dropna().sort_values()
ratio[ratio!=0].index[0]

result = df.iloc[ratio[ratio!=0].index[0]].title
print(result)

Join the BTS #PermissiontoDance Challenge only on YouTube #Shorts


8번 like 대비 dislike의 수가 가장 적은 영상은 무엇인가? (like, dislike 값이 0인경우는 제외한다)

In [ ]:
target = df.loc[(df.likes!=0)&(df.dislikes!=0)]
num = (target['dislikes']/target['likes']).sort_values().index[0]

result = df.iloc[num].title
print(result)

[줌터뷰] *최초공개* 사부작즈🐰🐶의 비공식 이름은 아이라인즈? 꿀조합 티키타카 가득한 NCT 127 도영&정우의 줌터뷰


9번 가장많은 트렌드 영상을 제작한 채널의 이름은 무엇인가? (날짜기준, 중복포함)

In [ ]:
result = df.loc[df.channelId == df.channelId.value_counts().index[0]].channelTitle.unique()[0]
print(result)

짤툰


10번 20회(20일)이상 인기동영상 리스트에 포함된 동영상의 숫자는?

In [ ]:
result = (df[['title','channelId']].value_counts()>=20).sum()
print(result)

40


#2유형

In [ ]:
import pandas as pd

X_test= pd.read_csv('/content/yemoonsaBigdata/datasets/Part3/404_x_test.csv')
X_train = pd.read_csv('/content/yemoonsaBigdata/datasets/Part3/404_x_train.csv')
y_train = pd.read_csv('/content/yemoonsaBigdata/datasets/Part3/404_y_train.csv')

#데이터 살펴보기
print(X_test.info())
print(X_train.info())
print(y_train.info())

print(X_test.isnull().sum())
print(X_train.isnull().sum())
print(y_train.isnull().sum())

#데이터 분류하기
COL_DEL = ['ID']
COL_NUM = ['Age','Work_Experience','Family_Size']
COL_CAT = ['Gender','Ever_Married','Graduated','Profession','Spending_Score']
COL_Y = ['Segmentation']

#데이터 분할하기
from sklearn.model_selection import train_test_split
X_tr , X_val, y_tr, y_val = train_test_split(X_train[COL_NUM+COL_CAT],
                                             y_train[COL_Y].values.ravel(),
                                             test_size=0.3,
                                             stratify=y_train[COL_Y].values.ravel())

#데이터 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[COL_NUM])

X_tr[COL_NUM] = scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.transform(X_test[COL_NUM])

#데이터 라벨링
label_mapping = {'A':0, 'B':1, 'C':2, 'D':3}

y_tr = [label_mapping[label] for label in y_tr]
y_val = [label_mapping[label] for label in y_val]

from sklearn.preprocessing import LabelEncoder

X = pd.concat([X_train[COL_CAT], X_test[COL_CAT]])

for col in COL_CAT:
    le = LabelEncoder()
    le.fit(X[col])
    X_tr[col] = le.transform(X_tr[col])
    X_val[col] = le.transform(X_val[col])
    X_test[col] = le.transform(X_test[col])

#모델 학습시키기
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(random_state=123)
modelRF.fit(X_tr, y_tr)

from xgboost import XGBClassifier
modelXGB = XGBClassifier(random_state=123)
modelXGB.fit(X_tr, y_tr)

#예측값 생성
y_val_predRF = modelRF.predict(X_val)
y_val_predXGB = modelXGB.predict(X_val)

#평가 지표 구하기
from sklearn.metrics import f1_score

scoreRF = f1_score(y_val, y_val_predRF, average='macro')
scoreXGB = f1_score(y_val, y_val_predXGB, average='macro')

print("Random Forest:", scoreRF)
print("XGBoost:", scoreXGB)

#점수가 더 높은 XGBoost 선정
def get_scores(model, X_tr, X_val, y_tr, y_val):
    y_tr_pred = model.predict(X_tr)
    y_val_pred = model.predict(X_val)
    tr_score = f1_score(y_tr, y_tr_pred, average='macro')
    val_score = f1_score(y_val, y_val_pred, average='macro')
    return f"train:{round(tr_score,4)}, valid{round(val_score,4)}"

print("Random Forest\n",get_scores(modelRF, X_tr, X_val, y_tr, y_val))
print("XGBoost\n",get_scores(modelXGB, X_tr, X_val, y_tr, y_val))

modelXGB2 = XGBClassifier(n_estimators=50, max_depth=3, min_child_weight=1, random_state=123)
modelXGB2.fit(X_tr, y_tr)

modelXGB3 = XGBClassifier(n_estimators=50, max_depth=3, min_child_weight=2, random_state=123)
modelXGB3.fit(X_tr, y_tr)

modelXGB4 = XGBClassifier(n_estimators=50, max_depth=5, min_child_weight=1, random_state=123)
modelXGB4.fit(X_tr, y_tr)

modelXGB5 = XGBClassifier(n_estimators=100, max_depth=5, min_child_weight=1, random_state=123)
modelXGB5.fit(X_tr, y_tr)

print('XGBoost2:', get_scores(modelXGB2, X_tr, X_val, y_tr, y_val))
print('XGBoost3:', get_scores(modelXGB3, X_tr, X_val, y_tr, y_val))
print('XGBoost4:', get_scores(modelXGB4, X_tr, X_val, y_tr, y_val))
print('XGBoost5:', get_scores(modelXGB5, X_tr, X_val, y_tr, y_val))

pred = modelXGB3.predict(X_test[COL_NUM+COL_CAT])
result = pd.DataFrame({'ID':X_test.ID, 'Segmentation':pred})
label_mapping2 = {0:'A', 1:'B', 2:'C', 3:'D'}

result['Segmentation'] = [label_mapping2[label] for label in result['Segmentation']]
print(result.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2178 entries, 0 to 2177
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2178 non-null   int64  
 1   Gender           2178 non-null   object 
 2   Ever_Married     2178 non-null   object 
 3   Age              2178 non-null   int64  
 4   Graduated        2178 non-null   object 
 5   Profession       2178 non-null   object 
 6   Work_Experience  2178 non-null   float64
 7   Spending_Score   2178 non-null   object 
 8   Family_Size      2178 non-null   float64
dtypes: float64(2), int64(2), object(5)
memory usage: 153.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6718 entries, 0 to 6717
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               6718 non-null   int64  
 1   Gender           6718 non-null   object 
 2   Ever_Married     6718 no

In [ ]:
#저장하기
result.to_csv('이마ㅓㄴ이ㅏㄻ.csv', index=False)

#3유형

##1. 단일표본 T-검정

In [ ]:
import pandas as pd
import scipy.stats as stats
from math import sqrt

data1 = pd.read_csv('/content/yemoonsaBigdata/datasets/supplement/trees.csv')
X = data1['Height'].mean()
print(round(X,2))

t_score, p_value = stats.ttest_1samp(data1['Height'], 75)
print(round(t_score,2))
print(round(p_value,4))

if p_value > 0.05:
    print('채택')
else:
    print('기각')

76.0
0.87
0.3892
채택


##2. 독립표본 T-검정

In [ ]:
import pandas as pd
import scipy.stats as stats
from math import sqrt

data2 = pd.read_csv('/content/yemoonsaBigdata/datasets/supplement/toothgrowth.csv')

stats.levene(data2.loc[data2.supp=='VC', 'len'], data2.loc[data.supp=='OJ','len'])

VC_data = data2.loc[data2.supp=='VC']
OJ_data = data2.loc[data2.supp=='OJ']

VC_X = VC_data['len'].mean()
OJ_X = OJ_data['len'].mean()

print(round(VC_X, 2), round(OJ_X, 2))

t_score, p_value = stats.ttest_ind(VC_data['len'], OJ_data['len'], equal_var=True)
print(round(t_score,2))
print(round(p_value,4))

if p_value>0.05:
    print('채택')
else:
    print('기각')

16.96 20.66
-1.92
0.0604
채택


##3. 쌍체표본 T-검정

In [ ]:
import pandas as pd
import scipy.stats as stats
from math import sqrt

a = pd.read_csv('/content/yemoonsaBigdata/datasets/supplement/insectsprays.csv')
print(a.head(),'\n')

diff = a['after_spr'] - a['before_spr']
m_d = diff.mean()
print('차이의 표본평균:', round(m_d,2),'\n')

t_score, p_value = stats.ttest_rel(a['before_spr'], a['after_spr'], alternative='greater')
print('t_score:', round(t_score,2))
print('p_value:', round(p_value,4),'\n')

if p_value>0.05:
    print('채택')
else:
    print('기각')

   before_spr  after_spr
0          10          0
1           7          1
2          20          7
3          14          2
4          14          3 

차이의 표본평균: -12.0 

t_score: 14.89
p_value: 0.0 

기각


## 4. 일원분산분석

In [ ]:
import pandas as pd
import scipy.stats as stats

b = pd.read_csv('/content/yemoonsaBigdata/datasets/supplement/iris.csv')

result = b.loc[b['species']=='versicolor','sepal_width'].mean()
print(round(result,2))

X1 = b.loc[b['species']=='setosa','sepal_width']
X2 = b.loc[b['species']=='versicolor','sepal_width']
X3 = b.loc[b['species']=='virginica','sepal_width']

test_result = stats.f_oneway(X1,X2,X3)
print(round(test_result.statistic,2))

p_value = test_result.pvalue
print(round(p_value,4))

if p_value > 0.05:
    print('채택')
else:
    print('기각')

2.77
49.16
0.0
기각


##5. 카이제곱검정 (적합도 검정)

In [ ]:
import pandas as pd
import scipy.stats as stats

c = pd.read_csv('/content/yemoonsaBigdata/datasets/supplement/cellphone.csv')

count_df = c.groupby('제조사').count()
print(count_df)
print('-'*80)

result = count_df.loc['삼성','id'] - count_df.loc['애플','id']
print(result)
print('-'*80)

total_sum = len(c)
expected = [int(total_sum*0.1),
            int(total_sum*0.6),
            int(total_sum*0.3)]
print(expected)
print('-'*80)

observed = count_df['id'].values
test_result = stats.chisquare(observed, f_exp=expected)
result = round(test_result.statistic,2)
print(result)
print('-'*80)

p_value = test_result.pvalue
print(p_value)

if p_value > 0.05 :
    print('채택')
else:
    print('기각')

      id
제조사     
기타     5
삼성   117
애플    78
--------------------------------------------------------------------------------
39
--------------------------------------------------------------------------------
[20, 120, 60]
--------------------------------------------------------------------------------
16.73
--------------------------------------------------------------------------------
0.00023345995371416804
기각


##6. 카이제곱검정 (독립성 검정)

In [ ]:
import pandas as pd
import scipy.stats as stats

d = pd.read_csv('/content/yemoonsaBigdata/datasets/supplement/student.csv')

table = pd.crosstab(d['단과대학'], d['수강 과목 수'])
print(table)
print('-'*80)

result = table[table['6개 이상']==table['6개 이상'].max()].index[0]
print(result)
print('-'*80)

X1 = table.loc['경영대학',:]
X2 = table.loc['자연대학',:]
X3 = table.loc['공과대학',:]

test_result = stats.chi2_contingency([X1,X2,X3])
result = test_result.statistic
print(round(result,2))
print('-'*80)

p_value = test_result.pvalue
print(round(p_value,4))
print('-'*80)

if p_value > 0.05:
    print('채택')
else:
    print('기각')

수강 과목 수  4개 이하  5개  6개 이상
단과대학                     
경영대학        12  78     30
공과대학        30  42     28
자연대학        30  42      8
--------------------------------------------------------------------------------
경영대학
--------------------------------------------------------------------------------
30.13
--------------------------------------------------------------------------------
0.0
--------------------------------------------------------------------------------
기각
